In [ ]:
#For this assignment I'll be replicating the code from the notebook
#First we'll grab our data, I'm going to make a variable for the web link
#and one for the local file I'll be using

In [4]:
import pandas as pd

local = '../data/nyc/nyc-rent-2016.csv'
web = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(local)
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,...,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,...,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Next, I'm going to import pandas_profiling, the following line is required outside a colab environment
#--upgrade insures we have the latest version
!pip install --upgrade pandas-profiling

Requirement already up-to-date: pandas-profiling in c:\programdata\anaconda3\lib\site-packages (2.1.0)


In [6]:
#A quick import and version check to make sure we're up to date
import pandas_profiling as pdp
pdp.__version__

'2.1.0'

In [8]:
#Now we'll store a report in a variable, then save it to an HTML document that will make things easy to look at
profile = df.profile_report()
profile.to_file(output_file="report-1.html")

In [12]:
#Next, we're going to split our dataset by time. We know from our previous .head() call that created is a date
#Let's make sure we can use it like one
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [14]:
df['created'].describe()

count                   48300
unique                  47643
top       2016-06-12 13:20:45
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [15]:
#Seeing we have 3 different months, we'll split train and test between the earliest two, and the latest month
#First, we make the month easy to access
df['month'] = df['created'].dt.month

In [16]:
#Now, we can see exactly which numbers we're working with
df['month'].describe()

count    48300.000000
mean         5.014928
std          0.824557
min          4.000000
25%          4.000000
50%          5.000000
75%          6.000000
max          6.000000
Name: month, dtype: float64

In [17]:
#Here we use filtering to split our dataframe into a train and test set
train = df[df['month']<=5]
test = df[df['month']==6]

In [18]:
#Here we compare the total counts in each to make sure they still have the right number of entries
train['created'].describe()

count                   31515
unique                  31116
top       2016-05-02 03:41:36
freq                        3
first     2016-04-01 22:12:41
last      2016-05-31 23:10:48
Name: created, dtype: object

In [19]:
test['created'].describe()

count                   16785
unique                  16527
top       2016-06-05 01:32:36
freq                        3
first     2016-06-01 01:10:37
last      2016-06-29 21:41:47
Name: created, dtype: object

In [24]:
#Next, we'll try and get a little better than randomly guessing prediction results
#We'll start by finding the mean in our train set's price
train['price'].mean()

3432.7534190068222

In [22]:
#Next, we'll grab the first 10 entries in our test set
first10 = test[['price']].head(10)
first10

,price
0,3000
1,5465
11,1675
14,3050
24,7400
34,2300
39,2396
47,1900
58,2000
81,2625


In [25]:
#Now we make a column of "predictions" using the mean for train
first10['predicted'] = [3432] * 10
first10

,price,predicted
0,3000,3432
1,5465,3432
11,1675,3432
14,3050,3432
24,7400,3432
34,2300,3432
39,2396,3432
47,1900,3432
58,2000,3432
81,2625,3432


In [26]:
#Now we'll find how far off we were each guess, or our 'error'
first10['error'] = first10['price']-first10['predicted']
first10

,price,predicted,error
0,3000,3432,-432
1,5465,3432,2033
11,1675,3432,-1757
14,3050,3432,-382
24,7400,3432,3968
34,2300,3432,-1132
39,2396,3432,-1036
47,1900,3432,-1532
58,2000,3432,-1432
81,2625,3432,-807


In [27]:
#Next we'll find the mean of the absolute values of our error column, to account for positive and negative numbers
first10['error'].abs().mean()

1451.1

In [28]:
#Next, we'll do everything all the previous lines have been doing, with just a single import and function call
from sklearn.metrics import mean_absolute_error

mean_absolute_error(first10['price'],first10['predicted'])

1451.1

In [33]:
#Finally, we'll create prediction and real price variables for the whole dataset, and run the function on those
y_test = test['price']
y_pred = [train['price'].mean()] * len(y_test)

In [30]:
#confirming they're the same length
assert len(y_test)==len(y_pred)

In [34]:
mean_absolute_error(y_test,y_pred)

1052.6798017842948